# Qfold cnot 7x2 Qiskit

<a id = 'index'></a>

* **Step 1.** - [Initial setup](#set)

* **Step 2.** - [Defining the circuit](#nc)

* **Step 3.** - [Optimizing the circuit](#op)

* **Step 4.** - [Running in real device and data treatment](#real)

* **Step 5.** - [Tests with different inputs](#oin)

<a id = 'set'></a>

## Initial Setup

To achieve results comparable to the ones present in paper *Compiling quantamorphisms for the IBM Q-Experience*, ensure the application of the correct imports and versions. 

In [ ]:
import qiskit
import qiskit.tools.jupyter
%qiskit_version_table

In [ ]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi

In [ ]:
# these imports are essential since the new circuit section
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

In [ ]:
# this import is essential since the simulation section
from qiskit import Aer

In [ ]:
# these imports are essential since the optimization section
from qiskit import IBMQ
from qiskit.tools.monitor import backend_monitor, backend_overview
from qiskit.compiler import transpile

In [ ]:
# this import is essential since run in real device section
from qiskit.tools.visualization import plot_histogram

In [ ]:
# this is essential in the PyZX section
import pyzx as zx

In [ ]:
# this is essential in the ignis section
from qiskit.ignis.mitigation.measurement import ( complete_meas_cal, CompleteMeasFitter, MeasurementFilter )

#### Important/Useful functions 

In [ ]:
def circuit_inf(quantum_circuit):
    circuit_information={}
    
    # total number of operations in the circuit. no unrolling is done.
    circuit_size = quantum_circuit.size()
    circuit_information['size']=circuit_size
    
    # depth of circuit (number of ops on the critical path)
    circuit_depth = quantum_circuit.depth()
    circuit_information['depth']=circuit_depth
    
    # number of unentangled subcircuits in this circuit.
    # each subcircuit can in principle be executed on a different quantum processor!
    circuit_tensor = quantum_circuit.num_tensor_factors()
    circuit_information['tensor factors']= circuit_tensor
    
    # a breakdown of operations by type
    circuit_count = quantum_circuit.count_ops()
    circuit_information['operations']=circuit_count
    return circuit_information

In [ ]:
def running_circuit(circuit, backend, shots=1024):
    job_run = execute(circuit, backend, shots=shots)
    jobID_run = job_run.job_id()

    result_run = job_run.result()
    counts_run = result_run.get_counts(circuit)
    
    return jobID_run, counts_run

In [ ]:
def sum_the_target_0(counts_raw):
    k=counts_raw.keys()
    sum_counts_ok=sum_counts_bad=0
    lk=list(k)
    for x in lk:
        if x[3]=='0':
            sum_counts_ok=sum_counts_ok+counts_raw.get(x)
        else:
            sum_counts_bad=sum_counts_bad+counts_raw.get(x)
    return {'good': sum_counts_ok, 'bad': sum_counts_bad}

def sum_right(target, counts_raw):
    s = sum_the_target_0(counts_raw)
    if target==1:
        s['good_temp'] = s.pop('good')
        s['good']= s.pop('bad')
        s['bad']= s.pop('good_temp')
        
    return s

[back to top](#index)

<a id = 'nc'></a>

## New Circuit


Recall that the output of Quipper language goes thought the translator in quipperToQiskit. In this format, it is possible to define the circuit easily. 

Moreover, it is essential to simulate the experiment to see what are the ideal outputs. 

Qiskit swaps the least and the most significant qubits. Therefore, to keep conformity, there was a rearrangement of the least and most significant qubits. 

In other words, 
* qubit 0 is now qubit 4
* qubit 1 is now qubit 3
* qubit 2 holds
* qubit 3 is not qubut 1
* and qubit 4 is now qubit 0


In [ ]:
# number of qubits
n = 5
# create quantum register named 'qr'
qr = QuantumRegister(n, 'qr')
# create classical register named 'cr'
cr = ClassicalRegister(n, 'cr')

In [ ]:
#create quantum circuit
qc= QuantumCircuit(qr,cr)

Go to the document `circuit_cnot_7x2_qiskit.txt`, select all the unitary gates, and copy to the following cell. 

In [ ]:
qc.draw(output='mpl', scale=0.5)

In [ ]:
circuit_inf(qc)

### Simulation

This simulation can run with Aer or BasicAer. 

In [ ]:
# add measure gates
m4 = QuantumCircuit(qr, cr)

m4.measure(qr[4],cr[4])
m4.measure(qr[1],cr[1])
m4.measure(qr[2],cr[2])
m4.measure(qr[3],cr[3])

m4.draw(output='mpl')

In [ ]:
qc_m = qc + m4

qc_m.draw(output='mpl')

In [ ]:
circuit_inf(qc_m)

In [ ]:
# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc_m, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

In [ ]:
counts_sim = result_sim.get_counts(qc_m)

In [ ]:
print(counts_sim)

[back to top](#index)

<a id = 'op'></a>

## Optimizing the circuit


The considerable volume of the circuit displayed points to its optimization. 


In [ ]:
# https://qiskit.org/documentation/install.html#access-ibm-quantum-systems
#
#provider = IBMQ.save_account('token')

provider = IBMQ.load_account()

In [ ]:
my_providers=IBMQ.providers()
print(my_providers)

In [ ]:
#  you may not have this access
my_provider_academic = IBMQ.get_provider(hub='my_hub', group='my_group', project='my_project')

my_provider_academic.backends()

In [ ]:
my_provider_ibmq = IBMQ.get_provider(hub='ibm-q', group='open', project='main') 

my_provider_ibmq.backends()

In [ ]:
%qiskit_backend_overview

In [ ]:
backend_overview()

**Boebligen** - choosen because has hight T2 comparing to the others. 

In [ ]:
backend = my_provider_academic.get_backend('ibmq_boeblingen')

In [ ]:
qc_sim = transpile(qc_m, backend=backend)

In [ ]:
qc_sim.draw(output='mpl')

In [ ]:
circuit_inf(qc_sim)

In [ ]:
backend_monitor(backend)

### IBM Q Transpiler

One trivial approach is to apply the IBM Q transpiler. 

In [ ]:
optimized_0 = transpile(qc_m, backend=backend, optimization_level=0)
circuit_inf(optimized_0)

In [ ]:
optimized_1 = transpile(qc_m, backend=backend, optimization_level=1)
circuit_inf(optimized_1)

In [ ]:
optimized_2 = transpile(qc_m, backend=backend, optimization_level=2)
circuit_inf(optimized_2)

In [ ]:
optimized_3 = transpile(qc_m, backend=backend, optimization_level=3)
circuit_inf(optimized_3)

<div class="alert alert-block alert-info">
try with optimization 2 and 3

<p>2 has less depth.</p>

<p>3 has less cnot.</p>
</div>

### PyZX

Since this optimization was insufficient, the circuit ended rewritten with PyZX. 

In [ ]:
my_qc = zx.Circuit.from_quipper_file("circuit_cnot_7x2_quipper_A.txt")

zx.draw(my_qc)

In [ ]:
print(my_qc.gates)

In [ ]:
print(my_qc.stats())

In [ ]:
mg = my_qc.to_graph()
print(mg)

In [ ]:
zx.simplify.full_reduce(mg)
zx.draw(mg)

In [ ]:
print(mg)

In [ ]:
mg.normalise()
zx.draw(mg)

In [ ]:
print(mg)

In [ ]:
mc = zx.extract.streaming_extract(mg.copy(), True)
zx.draw(mc)

In [ ]:
print(mc)

In [ ]:
# Turn graph back into circuit
mc2 = zx.extract.streaming_extract(mg).to_basic_gates()

In [ ]:
print(mc2.stats())

In [ ]:
mc3 = zx.optimize.full_optimize(mc2)
print(mc3.stats())

In [ ]:
print(mc3.to_quipper())

In [ ]:
f = open("quipper_pyzx.txt", "w")
f.write(mc3.to_quipper())
f.close()

* Open the `quipperToQiskit.gawk` file; 
* In line 2 change "qc" to "qc_pyzx";
* Save;
* Run the command line:
```
awk -f quipperToQiskit.gawk circuit_cnot_7x2_quipper_A.txt > circuit_cnot_7x2_qiskit_pyzx.txt 
```

In [ ]:
qc_pyzx = QuantumCircuit(qr, cr)

Go to the document `circuit_cnot_7x2_qiskit_pyzx.txt`, select all the unitary gates, and copy to the following cell. 

In [ ]:
qc_pyzx.draw(output='mpl')

In [ ]:
circuit_inf(qc_pyzx)

In [ ]:
qc_pyzx = qc_pyzx+m4

In [ ]:
id_temp, counts_pyzx_sim = running_circuit(qc_pyzx, backend_sim)
print(counts_pyzx_sim)

In [ ]:
qc_sim_pyzx = transpile(qc_pyzx, backend=backend)

In [ ]:
circuit_inf(qc_sim_pyzx)

In [ ]:
qc_pyzx_o2 = transpile(qc_pyzx, backend=backend, optimization_level=2)
circuit_inf(qc_pyzx_o2)

In [ ]:
qc_pyzx_o3 = transpile(qc_pyzx, backend=backend, optimization_level=3)
circuit_inf(qc_pyzx_o3)

[back to top](#index)

<a id='real'> </a>

## Running in the real device and data treatment


After reaching optimization, the IBM Q Experience Ignis module ensures the filtration of the results. 

Furthermore, the target as the most relevant qubit in the program leads to analyzing only the target result. 

In [ ]:
%qiskit_job_watcher

In [ ]:
shots=1024

Save the job Id value to recover the job information later.

In [ ]:
id_run, counts_dev = running_circuit(qc_m, backend, shots)
print(id_run)

In [ ]:
#id_run=''
job = backend.retrieve_job(id_run)
result_run= job.result()
counts_dev= result_run.get_counts()

print(counts_dev)

In [ ]:
id_opt2, counts_opt2 = running_circuit(optimized_2, backend, shots)
print(id_opt2)

In [ ]:
#id_opt2=''
job = backend.retrieve_job(id_opt2)
result_opt2= job.result()
counts_opt2= result_opt2.get_counts()

print(counts_opt2)

In [ ]:
id_opt3, counts_opt3 = running_circuit(optimized_3, backend, shots)
print(id_opt3)

In [ ]:
#id_opt3=''
job = backend.retrieve_job(id_opt3)
result_opt3= job.result()
counts_opt3= result_opt3.get_counts()

print(counts_opt3)

In [ ]:
id_pyzx, counts_pyzx = running_circuit(qc_pyzx, backend, shots)
print(id_pyzx, counts_pyzx)

In [ ]:
#id_pyzx=''
job = backend.retrieve_job(id_pyzx)
result_pyzx= job.result()
counts_pyzx= result_pyzx.get_counts()

print(counts_pyzx)

In [ ]:
id_pyzx_2, counts_pyzx_2 = running_circuit(qc_pyzx_o2, backend)
print(id_pyzx_2)

In [ ]:
#id_pyzx_2=''
job = backend.retrieve_job(id_pyzx_2)
result_pyzx_2= job.result()
counts_pyzx_2= result_pyzx_2.get_counts()

print(counts_pyzx_2)

In [ ]:
id_pyzx_3, counts_pyzx_3 = running_circuit(qc_pyzx_o3, backend)
print(id_pyzx_3)

In [ ]:
#id_pyzx_3=''
job = backend.retrieve_job(id_pyzx_3)
result_pyzx_3= job.result()
counts_pyzx_3= result_pyzx_3.get_counts()

print(counts_pyzx_3)

In [ ]:
leg = ['simulation', 'run in real device', 'transpiler with optimization 2', 'traspiler with optimization 3', 'optimization with compiler PyZX', 'optimization with compiler PyZX and 2', 'optimization with compiler PyZX and 3']
colors = ['#061727', '#003a6d', '#00539a', '#1192e8','#33b1ff','#82cfff','#e5f6ff']
plot_histogram([counts_sim, counts_dev,counts_opt2,counts_opt3,counts_pyzx,counts_pyzx_2,counts_pyzx_3], number_to_keep = 1, color=colors, legend = leg, figsize=(11, 5))

### Ignis

In [ ]:
# Generate the calibration circuits
qr_ignis = QuantumRegister(5)
meas_calibs, state_labels = complete_meas_cal(qubit_list=[0,1,2,3,4], qr=qr_ignis, circlabel='mcal')

In [ ]:
job_ignis = execute(meas_calibs, backend=backend)
cal_results = job_ignis.result()

jobID_run_ignis = job_ignis.job_id()
print('JOB ID: {}'.format(jobID_run_ignis))

In [ ]:
#id_ignis=''
job_ignis = backend.retrieve_job(id_ignis)
cal_results= job_ignis.result()

In [ ]:
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
# Plot the calibration matrix
meas_fitter.plot_calibration()

In [ ]:
# What is the measurement fidelity?
print("Average Measurement Fidelity: %f" % meas_fitter.readout_fidelity())

In [ ]:
# Get the filter object
meas_filter = meas_fitter.filter

In [ ]:
job_pyzx_3 = backend.retrieve_job(id_pyzx_3)
result_pyzx_3= job_pyzx_3.result()

In [ ]:
mitigated_results_py3 = meas_filter.apply(result_pyzx_3)
mitigated_counts_py3 = mitigated_results_py3.get_counts(0)

In [ ]:
print(mitigated_counts_py3)

In [ ]:
leg = ['simulation', 'run in real device', 'optimization with compiler PyZX and 3', 'mittigation']
colors = ['#061727', '#003a6d', '#82cfff','#e5f6ff']
plot_histogram([counts_sim, counts_dev,counts_pyzx, mitigated_counts_py3], number_to_keep = 1, color=colors, legend = leg, figsize=(11, 5))

### Find just the ones where the target qubit holds |0>

In [ ]:
counts_sim_0 = sum_right(0, counts_sim)
print(counts_sim_0)

In [ ]:
counts_dev_0 = sum_right(0, counts_dev)
print(counts_dev_0)

In [ ]:
counts_opt2_0 = sum_right(0, counts_opt2)
print(counts_opt2_0)

In [ ]:
counts_opt3_0 = sum_right(0, counts_opt3)
print(counts_opt3_0)

In [ ]:
counts_pyzx_0 = sum_right(0, counts_pyzx)
print(counts_pyzx_0)

In [ ]:
counts_pyzx2_0 = sum_right(0, counts_pyzx_2)
print(counts_pyzx2_0)

In [ ]:
counts_pyzx3_0 = sum_right(0, counts_pyzx_3)
print(counts_pyzx3_0)

In [ ]:
mitigated_counts_0= sum_right(0, mitigated_counts_py3)
print(mitigated_counts_0)

In [ ]:
leg = ['simulation', 'run in real device', 'optimization with compiler PyZX and 3 and mittigation']
colors = ['#061727', '#003a6d', '#1192e8']
plot_histogram([counts_sim_0, counts_dev_0, mitigated_counts_0], title='input=|0000>', color=colors, legend = leg, figsize=(5, 5))

[back to top](#index)

<a id = 'oin'></a>

## Tests with different inputs

To obtain a faithful experience, tests with different inputs are essential.

When the controls are |101> the target should change (the initial target value is |1>)

In [ ]:
qc_init_1011 = QuantumCircuit(qr,cr)

In [ ]:
qc_init_1011.x(qr[2])
qc_init_1011.x(qr[4])
qc_init_1011.x(qr[1])


qc_init_1011.draw(output='mpl')

In [ ]:
qc_1011 = qc_init_1011 + qc_m

qc_1011.draw(output='mpl')

In [ ]:
qc_1011_pyzx = qc_init_1011 + qc_pyzx

qc_1011_pyzx.draw(output='mpl')

In [ ]:
id_temp, counts_sim_1011 = running_circuit(qc_1011, backend_sim)
print(counts_sim_1011)

In [ ]:
id_temp, counts_sim_1011_p = running_circuit(qc_1011_pyzx, backend_sim)
print(counts_sim_1011_p)

In [ ]:
id_run_1011, counts_dev_1011 = running_circuit(qc_1011, backend)
print(id_run_1011)

In [ ]:
#id_run_1011=''
job = backend.retrieve_job(id_run_1011)
result_run_1011= job.result()
counts_dev_1011= result_run_1011.get_counts()

print(counts_dev_1011)

In [ ]:
id_pyzx_1011, counts_pyzx_1011 = running_circuit(qc_1011_pyzx, backend)
print(id_pyzx_1011)

In [ ]:
#id_pyzx_1011=''
job = backend.retrieve_job(id_pyzx_1011)
result_pyzx_1011= job.result()
counts_pyzx_1011= result_pyzx_1011.get_counts()

print(counts_pyzx_1011)

In [ ]:
qc_pyzx_o3_1011 = transpile(qc_1011_pyzx, backend=backend, optimization_level=3)

In [ ]:
id_temp, counts_sim_1011_p3 = running_circuit(qc_pyzx_o3_1011, backend_sim)
print(counts_sim_1011_p3)

In [ ]:
id_pyzx3_1011, counts_pyzx3_1011 = running_circuit(qc_pyzx_o3_1011, backend)
print(id_pyzx3_1011)

In [ ]:
#id_pyzx3_1011=''
job = backend.retrieve_job(id_pyzx3_1011)
result_pyzx3_1011= job.result()
counts_pyzx3_1011= result_pyzx3_1011.get_counts()

print(counts_pyzx3_1011)

In [ ]:
qc_pyzx_o2_1011 = transpile(qc_1011_pyzx, backend=backend, optimization_level=2)

In [ ]:
id_temp, counts_sim_1011_p2 = running_circuit(qc_pyzx_o2_1011, backend_sim)
print(counts_sim_1011_p2)

In [ ]:
id_pyzx2_1011, counts_pyzx2_1011 = running_circuit(qc_pyzx_o2_1011, backend)
print(id_pyzx2_1011)

In [ ]:
#id_pyzx2_1011=''
job = backend.retrieve_job(id_pyzx2_1011)
result_pyzx2_1011= job.result()
counts_pyzx2_1011= result_pyzx2_1011.get_counts()

print(counts_pyzx2_1011)

In [ ]:
leg = ['simulation', 'run in real device','optimization with compiler PyZX','optimization with compiler PyZX and 2', 'optimization with compiler PyZX and 3']
colors = ['#061727', '#003a6d', '#1192e8','#82cfff','#e5f6ff']
plot_histogram([counts_sim_1011, counts_dev_1011, counts_pyzx_1011, counts_pyzx2_1011, counts_pyzx3_1011], target_string='10100', number_to_keep=1,color=colors, legend = leg, figsize=(11, 5), title='intup |101> with target input |1>')

In [ ]:
job_pyzx_2_1011 = backend.retrieve_job(id_pyzx2_1011)
result_pyzx2_1011= job_pyzx_2_1011.result()

In [ ]:
mitigated_results_py2_1011 = meas_filter.apply(result_pyzx2_1011)
mitigated_counts_py2_1011 = mitigated_results_py2_1011.get_counts(0)

In [ ]:
print(mitigated_counts_py2_1011)

In [ ]:
job_pyzx_1011 = backend.retrieve_job(id_pyzx_1011)
result_pyzx_1011= job_pyzx_1011.result()

In [ ]:
mitigated_results_py_1011 = meas_filter.apply(result_pyzx_1011)
mitigated_counts_py_1011 = mitigated_results_py_1011.get_counts(0)

In [ ]:
print(mitigated_counts_py_1011)

In [ ]:
leg = ['simulation', 'run in real device','optimization with compiler PyZX', 'mittigation', 'optimization with compiler PyZX and 2', 'mittigation 2']
colors = ['#061727', '#003a6d', '#00539a', '#1192e8','#33b1ff','#82cfff']
plot_histogram([counts_sim_1011, counts_dev_1011, counts_pyzx_1011,mitigated_counts_py_1011, counts_pyzx2_1011,mitigated_counts_py2_1011 ], target_string='10100', number_to_keep = 1, color=colors, legend = leg, figsize=(11, 5), title='intup |101> with target input |1>')

In [ ]:
counts_sim_2 = sum_right(0, counts_sim_1011)
print(counts_sim_2)

In [ ]:
counts_dev_2 = sum_right(0, counts_dev_1011)
print(counts_dev_2)

In [ ]:
counts_pyzx_2 = sum_right(0, counts_pyzx_1011)
print(counts_pyzx_2)

In [ ]:
counts_pyzx2_2 = sum_right(0, counts_pyzx2_1011)
print(counts_pyzx2_2)

In [ ]:
counts_pyzx3_2 = sum_right(0, counts_pyzx3_1011)
print(counts_pyzx3_2)

In [ ]:
mitigated_counts_2= sum_right(0, mitigated_counts_py_1011)
print(mitigated_counts_2)

In [ ]:
mitigated_counts_2_2= sum_right(0, mitigated_counts_py2_1011)
print(mitigated_counts_2_2)

In [ ]:
leg = ['simulation', 'run in real device', 'optimization with compiter PyZX and mittigation']
colors = ['#061727', '#003a6d', '#1192e8']
plot_histogram([counts_sim_2, counts_dev_2,  mitigated_counts_2], title='input=|1011>', color=colors, legend = leg, figsize=(5, 5))

#### other important simulations 

In [ ]:
qubit_controls = 3
qubit_target = 1

total_qubits = 3+1

number_inputs = 2**total_qubits

data = np.arange(number_inputs)
data_input=[]

for i in data:
    data_input.append(bin(i)[2:].zfill(total_qubits))

print(data_input)

In [ ]:
for i in data_input:
    print('init circuit: ', i[::-1])
    count=0
    qcircuit = QuantumCircuit(qr,cr)
    for y in i:
        if y == '1':
            qcircuit.x(qr[count+1])
        count = count+1
    qcircuit = qcircuit + qc_m
    id_temp, counts_temp = running_circuit(qcircuit, backend_sim)
    print('output:       ', list(counts_temp.keys())[0][:-1])

In [ ]:
for i in data_input:
    print('init circuit: ', i[::-1])
    count=0
    qcircuit = QuantumCircuit(qr,cr)
    for y in i:
        if y == '1':
            qcircuit.x(qr[count+1])
        count = count+1
    qcircuit = qcircuit + qc_pyzx
    id_temp, counts_temp = running_circuit(qcircuit, backend_sim)
    print('output:       ', list(counts_temp.keys())[0][:-1])

[back to top](#index)